<a href="https://colab.research.google.com/github/avyaktawrat/Evaluat-inator/blob/master/Auto_encoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
movies = pd.read_csv('https://github.com/avyaktawrat/Evaluat-inator/raw/master/data/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('https://github.com/avyaktawrat/Evaluat-inator/raw/master/data/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('https://github.com/avyaktawrat/Evaluat-inator/blob/master/data/ratings.dat?raw=true', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

movies.columns = ['movieID', 'title', 'genres']
users.columns = ['userId',	'gender',	'age_desc',	'occ_desc', 'zipcode']
ratings.columns = ['userId',	'movieId',	'rating',	'timestamp']

In [3]:
movies.head()

,movieID,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users.head()

,userId,gender,age_desc,occ_desc,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [0]:
from sklearn.model_selection import KFold

In [8]:
cv = KFold(5, True, random_state=42)
cv

KFold(n_splits=5, random_state=42, shuffle=True)

In [0]:
ratings_np = ratings.to_numpy()

In [10]:
for train, test in cv.split(ratings_np):
  training_set = ratings_np[train]
  test_set = ratings_np[test]
  break
print('train: %s, test: %s' % (training_set, test_set.shape))
training_set.shape

train: [[        1       661         3 978302109]
 [        1       914         3 978301968]
 [        1      3408         4 978300275]
 ...
 [     6040      1094         5 956704887]
 [     6040      1096         4 956715648]
 [     6040      1097         4 956715569]], test: (200042, 4)


(800167, 4)

In [0]:
# Preparing the training set and the test set
# training_set = pd.read_csv('u1.base', delimiter = '\t')
# print(training_set.head())
# training_set = np.array(training_set, dtype = 'int')
# test_set = pd.read_csv('u1.test', delimiter = '\t')
# test_set = np.array(test_set, dtype = 'int')

In [12]:
training_set.shape

(800167, 4)

In [0]:
np.save('training_set.npy',training_set)
np.save('test_set.npy',test_set)

In [0]:
# training_set = np.load('training_set.npy')

In [0]:
# test_set = np.load('test_set.npy')

In [0]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [0]:
# Why doing this ? Because we want to create a new structure of data, having the shape of a 2d array where:

#    the rows are the users,
#    the columns are the movies,
#    the cells are the ratings.

In [0]:
def convert(data):
    new_data = []
    for id_users in range(1,nb_users+1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        #all rows such that first coloumn is id_users
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [0]:
training_set = convert(training_set)

In [0]:
test_set = convert(test_set)

In [0]:
np.save('training_set_proc.npy',training_set)
np.save('test_set_proc.npy',test_set)
# training_set = np.load('training_set_proc.npy')
# test_set = np.load('test_set_proc.npy')

In [0]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [0]:
class SAE(nn.Module):
    def __init__(self, ):
        # super function is used to use classes of parent class
        super(SAE,self).__init__()
        # by this we can get all the inherited classes of nn.Module
        # first argument is the features, second is the the number of units
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20,10) #second layer has 10 neurons
        self.fc3 = nn.Linear(10,20)
        self.fc4 = nn.Linear(20,nb_movies)
        self.activation = nn.Sigmoid()
        self.activation_t = nn.Tanh()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation_t(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [0]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.Adam(sae.parameters(),lr=0.01,weight_decay=0.5)
#decay is used for regulating the steps of the optimizer

In [0]:
#Training the AutoEncoders

In [0]:
nb_epoch = 200
loss_iteration = np.zeros(nb_epoch)

In [27]:
for epoch in range(1,nb_epoch+1):
    train_loss = 0
    #number of users who at least rated one movie to reduce the computation
    s = 0. #RMSE needs a float
    for id_user in range(nb_users):
        input = Variable(training_set[id_user,:]).unsqueeze(0) #dimension for a batch is also added here on the 0th axis
        #weights in this code are updated after each i/p vector
        target = input.clone() #target is the input
        if torch.sum(target.data > 0) > 0:
            # if the user has rated atleast one movie
            output = sae(input) #predicted ratings are returned for this particular user
            target.require_grad = False #gradient is a clone of i/p so now its gradient won't be calculated
            output[target == 0] = 0 #These are zeros so they don't need to be included in the computation
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) #denominator should not be zero
            # it is the average of the error with non-zero ratings
            loss.backward()
            train_loss = train_loss + np.sqrt(loss.detach()*mean_corrector) #loss.data[0] is the loss value
            s= s+1
            optimizer.step()
            # backward decides the direction(increased or decreased) of the weights and optimizer decides their intensities

    loss_iteration[epoch-1] = train_loss/s
    print('epoch: '+str(epoch)+ ' loss: '+ str(train_loss/s)) #train loss divided by number of users with atleast one rating
    

epoch: 1 loss: tensor(1.3380)
epoch: 2 loss: tensor(1.0004)
epoch: 3 loss: tensor(0.9820)
epoch: 4 loss: tensor(0.9758)
epoch: 5 loss: tensor(0.9730)
epoch: 6 loss: tensor(0.9714)
epoch: 7 loss: tensor(0.9704)
epoch: 8 loss: tensor(0.9697)
epoch: 9 loss: tensor(0.9692)
epoch: 10 loss: tensor(0.9688)
epoch: 11 loss: tensor(0.9685)
epoch: 12 loss: tensor(0.9683)
epoch: 13 loss: tensor(0.9681)
epoch: 14 loss: tensor(0.9680)
epoch: 15 loss: tensor(0.9678)
epoch: 16 loss: tensor(0.9677)
epoch: 17 loss: tensor(0.9676)
epoch: 18 loss: tensor(0.9675)
epoch: 19 loss: tensor(0.9674)
epoch: 20 loss: tensor(0.9674)
epoch: 21 loss: tensor(0.9673)
epoch: 22 loss: tensor(0.9673)
epoch: 23 loss: tensor(0.9672)
epoch: 24 loss: tensor(0.9672)
epoch: 25 loss: tensor(0.9671)
epoch: 26 loss: tensor(0.9671)
epoch: 27 loss: tensor(0.9670)
epoch: 28 loss: tensor(0.9670)
epoch: 29 loss: tensor(0.9670)
epoch: 30 loss: tensor(0.9670)
epoch: 31 loss: tensor(0.9669)
epoch: 32 loss: tensor(0.9669)
epoch: 33 loss: t

In [28]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user,:]).unsqueeze(0) #because we are predicting ratings for that user
    target = Variable(test_set[id_user,:]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        s+=1.
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        pred_loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(pred_loss.detach()*mean_corrector)
print('test loss: '+ str(test_loss/s))

test loss: tensor(0.9310)


***Sanity Check***

In [0]:
user_rating = ratings.groupby('userId').size()
input = Variable(training_set[np.argmax(user_rating),:]).unsqueeze(0)
output = sae(input)
# rec_movie = movies['title'][movies['movieID'] == np.argmax(output.detach().numpy())]
# rec_movie_genres = movies['genres'][movies['movieID'] == np.argmax(output.detach().numpy())]
# rec_movie_genres

In [30]:
rec_movie_id = movies["movieID"]
id_movies = np.arange(1,max(rec_movie_id)+1)

rec_movie = pd.DataFrame(np.transpose(output.detach().numpy()), index=id_movies, columns=["Rating"])
rec_movie.sort_values('Rating',ascending=False)[:10]

,Rating
557,5.183513
787,4.841122
729,4.833947
3245,4.771235
2019,4.688835
2503,4.659563
2999,4.644917
53,4.599221
3307,4.570691
578,4.552640
